# Semiconductor Defect Detection by Hybrid Classical-Quantum Deep Learning

https://github.com/Yfyangd/CVPR2022

### 패키지 설치 

keras==2.6.0 / 
numpy==1.19.5 / 
matplotlib==3.3.2 / 
PennyLane==0.19.0 / 
tensorflow==2.3.1

In [4]:
pip install -r requirements.txt

  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
  Using cached numpy-1.19.5-cp39-cp39-win_amd64.whl (13.3 MB)
  Using cached matplotlib-3.3.2.tar.gz (37.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.3.1 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0)
ERROR: No matching distribution found for tensorflow==2.3.1


### quantum_circuit.py

In [1]:
import pennylane as qml

n_qubits = 10
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights_0, weight_1):
    qml.RX(inputs[0], wires=0)
    qml.RX(inputs[1], wires=1)
    qml.Rot(*weights_0, wires=0)
    qml.RY(weight_1, wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))

### Self_Attention.py

In [2]:
from tensorflow.keras.layers import LayerNormalization, Conv2D, Layer, Activation

class Self_Attention_Block(Layer):

    def __init__(self, filters, ratio):
        super(Self_Attention_Block, self).__init__()
        self.conv0 = Conv2D(1, (1, 1), strides=(1, 1), padding='same',
                           use_bias=False, activation=None)        
        self.softmax = Activation('softmax')
        self.conv1 = Conv2D(int(filters / ratio), (1, 1), strides=(1, 1), padding='same',
                           use_bias=False, activation=None)
        self.LN = LayerNormalization()
        self.conv2 = Conv2D(int(filters), (1, 1), strides=(1, 1), padding='same',
                           use_bias=False, activation=None)
        self.relu = Activation('relu')
        self.hard_sigmoid = Activation('hard_sigmoid')

    def call(self, inputs):
        x = self.conv0(inputs)
        self_attention = self.softmax(x)
        x = x * self_attention
        x = self.relu(self.LN(self.conv1(x)))
        excitation = self.hard_sigmoid(self.conv2(x))
        x = inputs * excitation
        return x



### Self_Proliferate_and_Attention.py

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, DepthwiseConv2D, Layer, Activation, add
from Self_Attention import Self_Attention_Block
from Self_Proliferate import Self_Proliferate_Block

class Self_Proliferate_and_Attention_Block(Layer):

    def __init__(self, dwkernel, strides, exp, out, ratio, use_se):
        super(Self_Proliferate_and_Attention_Block, self).__init__()
        self.strides = strides
        self.use_se = use_se
        self.conv = Conv2D(out, (1, 1), strides=(1, 1), padding='same',
                           activation=None, use_bias=False)
        self.relu = Activation('relu')
        self.depthconv1 = DepthwiseConv2D(dwkernel, strides, padding='same', depth_multiplier=ratio-1,
                                         activation=None, use_bias=False)
        self.depthconv2 = DepthwiseConv2D(dwkernel, strides, padding='same', depth_multiplier=ratio-1,
                                         activation=None, use_bias=False)
        for i in range(5):
            setattr(self, f"batchnorm{i+1}", BatchNormalization())
        self.SAB1 = Self_Proliferate_Block(exp, ratio, 1, 3)
        self.SAB2 = Self_Proliferate_Block(out, ratio, 1, 3)
        self.se = Self_Attention_Block(exp, ratio)

    def call(self, inputs):
        x = self.batchnorm1(self.depthconv1(inputs))
        x = self.batchnorm2(self.conv(x))

        y = self.relu(self.batchnorm3(self.SAB1(inputs)))
        if self.strides > 1:
            y = self.relu(self.batchnorm4(self.depthconv2(y)))
        if self.use_se:
            y = self.se(y)
        y = self.batchnorm5(self.SAB2(y))
        return add([x, y])



### Self_Proliferate.py

In [4]:
from tensorflow.keras.layers import Conv2D, Concatenate, DepthwiseConv2D, Layer, Activation
from math import ceil

class Self_Proliferate_Block(Layer):

    def __init__(self, out, ratio, convkernel, dwkernel):
        super(Self_Proliferate_Block, self).__init__()
        self.ratio = ratio
        self.out = out
        self.conv_out_channel = ceil(self.out * 1.0 / ratio)
        self.conv = Conv2D(int(self.conv_out_channel), (convkernel, convkernel), use_bias=False,
                           strides=(1, 1), padding='same', activation=None)
        self.depthconv = DepthwiseConv2D(dwkernel, 1, padding='same', use_bias=False,
                                         depth_multiplier=ratio-1, activation=None)
        self.concat = Concatenate()

    def call(self, inputs):
        x = self.conv(inputs)
        if self.ratio == 1:
            return x
        dw = self.depthconv(x)
        dw = dw[:, :, :, :int(self.out - self.conv_out_channel)]
        output = self.concat([x, dw])
        return output



### CosineAnnealing.py

In [5]:
import math
from keras import backend as K
from keras.callbacks import Callback


class CosineAnnealingScheduler(Callback):

    def __init__(self, T_max, eta_max, eta_min=0, verbose=0):
        super(CosineAnnealingScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.verbose = verbose

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')
        lr = self.eta_min + (self.eta_max - self.eta_min) * (1 + math.cos(math.pi * epoch / self.T_max)) / 2
        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: CosineAnnealingScheduler setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

#test

### CycleLoss.py

In [6]:
import tensorflow.keras.losses as kls # add kls code

class CircleLoss(kls.Loss):
    
    def __init__(self,gamma: int = 64,margin: float = 0.25,batch_size: int = None,reduction='auto',name=None):
        super().__init__(reduction=reduction, name=name)
        self.gamma = gamma
        self.margin = margin
        self.O_p = 1 + self.margin
        self.O_n = -self.margin
        self.Delta_p = 1 - self.margin
        self.Delta_n = self.margin
        if batch_size:
            self.batch_size = batch_size
            self.batch_idxs = tf.expand_dims(tf.range(0, batch_size, dtype=tf.int32), 1)  # shape [batch,1]

    def call(self, y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
        alpha_p = tf.nn.relu(self.O_p - tf.stop_gradient(y_pred))
        alpha_n = tf.nn.relu(tf.stop_gradient(y_pred) - self.O_n)
        y_true = tf.cast(y_true, tf.float32)
        y_pred = (y_true * (alpha_p * (y_pred - self.Delta_p)) 
                  + (1 - y_true) * (alpha_n * (y_pred - self.Delta_n))
                 ) * self.gamma
        return tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)

### Hybrid_CNN.py

In [7]:
import pennylane as qml
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D, BatchNormalization, Reshape, Activation, add
from Self_Proliferate_and_Attention import Self_Proliferate_and_Attention_Block
from quantum_circuit import qnode

class SPANet(tf.keras.Model):
    
    def __init__(self, classes):
        super(SPANet, self).__init__()
        self.classes = classes
        self.img_input = keras.layers.Input(shape=(1, 512, 512, 3))
        self.conv1 = Conv2D(16, (3, 3), strides=(2, 2), padding='same',
                            activation=None, use_bias=False)
        self.conv2 = Conv2D(960, (1, 1), strides=(1, 1), padding='same',
                            activation=None, use_bias=False)
        self.conv3 = Conv2D(1280, (1, 1), strides=(1, 1), padding='same',
                            activation=None, use_bias=False)
        self.conv4 = Conv2D(self.classes, (1, 1), strides=(1, 1), padding='same',
                            activation=None, use_bias=False)

        #Quantum Layer
        self.weight_shapes = {"weights_0": 3, "weight_1": 1}
        self.qlayer = qml.qnn.KerasLayer(qnode, self.weight_shapes, output_dim=self.classes)
        
        
        for i in range(3):
            setattr(self, f"batchnorm{i+1}", BatchNormalization())
        self.relu = Activation('relu')
        self.softmax = Activation('softmax')
        self.pooling = GlobalAveragePooling2D()

        self.dwkernels = [3, 3, 3, 5, 5, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, 5]
        self.strides = [1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]
        self.exps = [16, 48, 72, 72, 120, 240, 200, 184, 184, 480, 672, 672, 960, 960, 960, 960]
        self.outs = [16, 24, 24, 40, 40, 80, 80, 80, 80, 112, 112, 160, 160, 160, 160, 160]
        self.ratios = [2] * 16
        self.use_sa = [False, False, False, True, True, False, False, False,
                        False, True, True, True, False, True, False, True]
        for i, args in enumerate(zip(self.dwkernels, self.strides, self.exps, self.outs, self.ratios, self.use_sa)):
            setattr(self, f"Self_Proliferate_and_Attention_Block{i}", Self_Proliferate_and_Attention_Block(*args))

    def call(self, inputs):
        x = self.relu(self.batchnorm1(self.conv1(inputs)))
        # Iterate through Ghost Bottlenecks
        for i in range(16):
            x = getattr(self, f"Self_Proliferate_and_Attention_Block{i}")(x)
        x = self.relu(self.batchnorm2(self.conv2(x)))
        x = self.pooling(x)
        x = Reshape((1, 1, int(x.shape[1])))(x)
        x = self.relu(self.batchnorm3(self.conv3(x)))
        x = self.conv4(x)
        x = tf.squeeze(x, 1)
        x = tf.squeeze(x, 1)
        FC = tf.keras.layers.Dense(classes, activation="softmax")
        model = tf.keras.models.Sequential([x, self.qlayer, FC])
        model = tf.keras.Model(self.img_input,x)
        return output

